In [43]:
# Different variations of correlation pruning, SVD (num of components), and lin reg
# n_components = 2, the default, did the best

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.decomposition import PCA, TruncatedSVD

import pandas as pd
import numpy as np
import csv
import locale
from scipy.stats import randint
import joblib
locale.setlocale( locale.LC_ALL, '' )

'English_United States.1252'

In [37]:
def safe_log(x):
    return np.log(x + 1e-10)
#thanks to ChatGPT 3.5 for this function

In [18]:
train_set = pd.read_csv("train.csv")
test_set = pd.read_csv("test.csv")

In [20]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [22]:
housing = train_set.copy()

housing = housing.drop("PoolQC", axis=1)
housing = housing.drop("Fence", axis=1)
housing = housing.drop("MiscFeature", axis=1)
housing = housing.drop("FireplaceQu", axis=1)
housing = housing.drop("Alley", axis=1)

corr_matrix = housing.corr()
list_col = list(corr_matrix)
corr_matrix["SalePrice"]

['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']


C:\Users\user_02\AppData\Local\Temp\ipykernel_21044\4254063651.py:9: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = housing.corr()


Id              -0.021917
MSSubClass      -0.084284
LotFrontage      0.351799
LotArea          0.263843
OverallQual      0.790982
OverallCond     -0.077856
YearBuilt        0.522897
YearRemodAdd     0.507101
MasVnrArea       0.477493
BsmtFinSF1       0.386420
BsmtFinSF2      -0.011378
BsmtUnfSF        0.214479
TotalBsmtSF      0.613581
1stFlrSF         0.605852
2ndFlrSF         0.319334
LowQualFinSF    -0.025606
GrLivArea        0.708624
BsmtFullBath     0.227122
BsmtHalfBath    -0.016844
FullBath         0.560664
HalfBath         0.284108
BedroomAbvGr     0.168213
KitchenAbvGr    -0.135907
TotRmsAbvGrd     0.533723
Fireplaces       0.466929
GarageYrBlt      0.486362
GarageCars       0.640409
GarageArea       0.623431
WoodDeckSF       0.324413
OpenPorchSF      0.315856
EnclosedPorch   -0.128578
3SsnPorch        0.044584
ScreenPorch      0.111447
PoolArea         0.092404
MiscVal         -0.021190
MoSold           0.046432
YrSold          -0.028923
SalePrice        1.000000
Name: SalePr

In [23]:
housing_labels = pd.DataFrame(train_set["SalePrice"].copy())
housing = housing.drop("SalePrice", axis=1)
ids = test_set["Id"]

for k in range(len(list_col)):
    if corr_matrix["SalePrice"][k] < 0.5:
        housing = housing.drop(f"{list_col[k]}", axis=1)

In [24]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 48 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSZoning       1460 non-null   object
 1   Street         1460 non-null   object
 2   LotShape       1460 non-null   object
 3   LandContour    1460 non-null   object
 4   Utilities      1460 non-null   object
 5   LotConfig      1460 non-null   object
 6   LandSlope      1460 non-null   object
 7   Neighborhood   1460 non-null   object
 8   Condition1     1460 non-null   object
 9   Condition2     1460 non-null   object
 10  BldgType       1460 non-null   object
 11  HouseStyle     1460 non-null   object
 12  OverallQual    1460 non-null   int64 
 13  YearBuilt      1460 non-null   int64 
 14  YearRemodAdd   1460 non-null   int64 
 15  RoofStyle      1460 non-null   object
 16  RoofMatl       1460 non-null   object
 17  Exterior1st    1460 non-null   object
 18  Exterior2nd    1460 non-null

In [15]:
text_pipeline = make_pipeline(SimpleImputer(strategy="most_frequent"), OneHotEncoder(handle_unknown="ignore"))
num_norm_pipeline = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())
preprocess = ColumnTransformer([
    ("text", text_pipeline, make_column_selector(dtype_include=object))],
    remainder=num_norm_pipeline
)

In [29]:
lin_reg = Pipeline([("pre", preprocess), ("pca", TruncatedSVD()), ("reg", LinearRegression())])
lin_reg.fit(housing, housing_labels)

housing_predictions = lin_reg.predict(housing)
for k in range(10):
    print(f"Real value: {locale.currency(housing_labels.iloc[k].values[0], grouping=True)}, Prediction: {locale.currency(housing_predictions[k][0].round(2), grouping=True)}")

Real value: $208,500.00, Prediction: $227,008.76
Real value: $181,500.00, Prediction: $179,399.80
Real value: $223,500.00, Prediction: $232,768.88
Real value: $140,000.00, Prediction: $169,700.93
Real value: $250,000.00, Prediction: $284,263.44
Real value: $143,000.00, Prediction: $152,643.33
Real value: $307,000.00, Prediction: $264,789.15
Real value: $200,000.00, Prediction: $209,913.89
Real value: $129,900.00, Prediction: $167,287.07
Real value: $118,000.00, Prediction: $115,428.68


In [38]:
lin_rmse = mean_squared_error(safe_log(housing_labels), safe_log(housing_predictions), squared=False)
print(lin_rmse)

0.2015161779367964


In [50]:
param_distribs = {"pca__n_components": randint(low=2, high=10)}
rdm_search = RandomizedSearchCV(
    lin_reg, param_distributions=param_distribs, n_iter=10, cv=3, scoring="neg_root_mean_squared_error"
)
rdm_search.fit(housing, housing_labels)
final_lr_model = rdm_search.best_estimator_
print(final_lr_model["pca"])

TruncatedSVD(n_components=9)


In [60]:
housing_predictions2 = final_lr_model.predict(housing)
lin_rmse2 = mean_squared_error(housing_labels, housing_predictions2, squared=False)
print(lin_rmse2)

36785.51195237752


In [52]:
for k in range(10):
    print(f"Real value: {locale.currency(housing_labels.iloc[k].values[0], grouping=True)}, Prediction: {locale.currency(housing_predictions2[k][0].round(2), grouping=True)}")

Real value: $208,500.00, Prediction: $220,596.87
Real value: $181,500.00, Prediction: $164,584.36
Real value: $223,500.00, Prediction: $225,233.88
Real value: $140,000.00, Prediction: $200,252.39
Real value: $250,000.00, Prediction: $291,742.57
Real value: $143,000.00, Prediction: $144,734.53
Real value: $307,000.00, Prediction: $266,722.70
Real value: $200,000.00, Prediction: $217,718.28
Real value: $129,900.00, Prediction: $162,323.98
Real value: $118,000.00, Prediction: $114,910.67


In [46]:
# test_pred = final_lr_model.predict(test_set)
# with open('sacreddeer_house_submission_1.csv', 'w', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerow(["Id", "SalePrice"])
#     for k in range(len(ids)):
#         writer.writerow([ids[k], test_pred[k][0]])

In [47]:
# test_pred = lin_reg.predict(test_set)
# with open('sacreddeer_house_submission_2.csv', 'w', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerow(["Id", "SalePrice"])
#     for k in range(len(ids)):
#         writer.writerow([ids[k], test_pred[k][0]])

In [63]:
lin_reg0 = Pipeline([("pre", preprocess),
                     ("pca", TruncatedSVD(n_components=1)), ("reg", LinearRegression())])             
lin_reg2 = Pipeline([("pre", preprocess), 
                     ("pca", TruncatedSVD(n_components=3)), ("reg", LinearRegression())])
lin_reg3 = Pipeline([("pre", preprocess), 
                     ("pca", TruncatedSVD(n_components=4)), ("reg", LinearRegression())])
lin_reg4 = Pipeline([("pre", preprocess), 
                     ("pca", TruncatedSVD(n_components=5)), ("reg", LinearRegression())])
lin_reg5 = Pipeline([("pre", preprocess), 
                     ("pca", TruncatedSVD(n_components=6)), ("reg", LinearRegression())])


In [64]:
models = [lin_reg0, lin_reg, lin_reg2, lin_reg3, lin_reg4, lin_reg5, final_lr_model]
for k in range(len(models)):
    models[k].fit(housing, housing_labels)
    print(mean_squared_error(housing_labels, models[k].predict(housing), squared=False))


79414.51153158581
41552.185615124916
39823.434918528765
39787.041150819205
39733.719905974205
39621.47379127692
36834.09391895131


In [62]:
# test_pred = lin_reg3.predict(test_set)
# with open('sacreddeer_house_submission_3.csv', 'w', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerow(["Id", "SalePrice"])
#     for k in range(len(ids)):
#         writer.writerow([ids[k], test_pred[k][0]])

In [65]:
# test_pred = lin_reg0.predict(test_set)
# with open('sacreddeer_house_submission_4.csv', 'w', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerow(["Id", "SalePrice"])
#     for k in range(len(ids)):
#         writer.writerow([ids[k], test_pred[k][0]])

In [66]:
# test_pred = lin_reg2.predict(test_set)
# with open('sacreddeer_house_submission_5.csv', 'w', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerow(["Id", "SalePrice"])
#     for k in range(len(ids)):
#         writer.writerow([ids[k], test_pred[k][0]])